In [108]:
!pip install numpy


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [109]:
import os
import pandas as pd
import openpyxl
import matplotlib.pyplot as plt
import numpy as np



In [110]:
# 데이터 디렉토리 경로
directory_o = r'C:\Users\ch.kang\Downloads\신청페이지 들어간사람\q_l'
directory_x = r'C:\Users\ch.kang\Downloads\신청페이지 안들어간사람\q_l만'

# 모든 엑셀 파일의 경로를 가져오기
file_paths_o = [os.path.join(directory_o, file) for file in os.listdir(directory_o) if file.endswith('.xlsx')]
file_paths_x = [os.path.join(directory_x, file) for file in os.listdir(directory_x) if file.endswith('.xlsx')]



In [111]:
# 엑셀 파일을 읽어들여 데이터프레임으로 변환하여 리스트에 저장
dfs_o = []
for file_path_1 in file_paths_o:
    # 파일명에서 확장자를 제외한 부분 추출
    file_name = os.path.splitext(os.path.basename(file_path_1))[0]
    # 엑셀 파일을 읽어들여 데이터프레임으로 변환
    df_1 = pd.read_excel(file_path_1, dtype=str)
    # 'date' 열에 파일명에서 추출한 값을 추가
    df_1['date'] = file_name
    dfs_o.append(df_1)

dfs_x = []
for file_path_2 in file_paths_x:
    # 파일명에서 확장자를 제외한 부분 추출
    file_name = os.path.splitext(os.path.basename(file_path_2))[0]
    # 엑셀 파일을 읽어들여 데이터프레임으로 변환
    df_2 = pd.read_excel(file_path_2, dtype=str)
    # 'date' 열에 파일명에서 추출한 값을 추가
    df_2['date'] = file_name
    dfs_x.append(df_2)



In [112]:
# 리스트에 있는 모든 데이터프레임을 하나로 합치기
merged_df_o = pd.concat(dfs_o, ignore_index=True)
print(merged_df_o.columns)
merged_df_x = pd.concat(dfs_x, ignore_index=True)
print(merged_df_x.columns)


Index(['아이디', '스크롤여부', '클릭여부', '체류시간', 'date'], dtype='object')
Index(['아이디', '체류시간', '이탈유형', 'date'], dtype='object')


In [113]:
merged_df_o["신청진입여부"] = "Y"
merged_df_x["신청진입여부"] = "N"
# Sort the DataFrame by '체류시간' in ascending order
merged_df_o['체류시간'] = pd.to_numeric(merged_df_o['체류시간'], errors='coerce')
merged_df_x['체류시간'] = pd.to_numeric(merged_df_x['체류시간'], errors='coerce')

df_sorted_o = merged_df_o.sort_values(by='체류시간', ascending=True).reset_index(drop=True)
df_sorted_x = merged_df_x.sort_values(by='체류시간', ascending=True).reset_index(drop=True)

# Calculate the cumulative percentage for '체류시간'
df_sorted_o['누적%'] = (df_sorted_o.index + 1) / len(df_sorted_o) * 100
df_sorted_o['누적%'] = df_sorted_o['누적%'].round(1)
df_sorted_o

df_sorted_x['누적%'] = (df_sorted_x.index + 1) / len(df_sorted_x) * 100
df_sorted_x['누적%'] = df_sorted_x['누적%'].round(1)
df_sorted_x

merged_df = pd.concat([df_sorted_o, df_sorted_x], ignore_index=True)
print(len(df_sorted_o),len(df_sorted_x), len(merged_df))


# 체류시간을 10초 간격으로 나누어 새로운 열 추가
merged_df['10초 구간'] = (merged_df['체류시간'] // 10) * 10
merged_df.columns


13 985 998


Index(['아이디', '스크롤여부', '클릭여부', '체류시간', 'date', '신청진입여부', '누적%', '이탈유형',
       '10초 구간'],
      dtype='object')

In [114]:
df=merged_df.copy()
len(df)

998

In [115]:
def detect_outliers(data):
    """
    주어진 데이터에서 이상치를 판단하는 함수
    :param data: 이상치를 판단할 데이터 (시리즈 형태)
    :return: 이상치 여부를 나타내는 시리즈를 반환
    """
    # Z-점수 계산
    z_scores = np.abs((data - np.mean(data)) / np.std(data))
    # 이상치 판단
    outliers_mask = z_scores > 3
    return outliers_mask

# merged_df['체류시간'] 데이터를 사용하여 이상치 여부를 판단
outliers_mask = detect_outliers(df['체류시간'])

# 이상치 여부를 나타내는 열 추가
df['이상치'] = outliers_mask



In [116]:
df

,아이디,스크롤여부,클릭여부,체류시간,date,신청진입여부,누적%,이탈유형,10초 구간,이상치
0,1042732193.1712370545,N,N,2,20240408,Y,7.7,NaN,0,False
1,1819785872.1712371735,N,N,10,20240406,Y,15.4,NaN,10,False
2,1653590084.1712352132,N,N,15,20240406,Y,23.1,NaN,10,False
3,2084490160.1712804830,Y,N,17,20240411,Y,30.8,NaN,10,False
4,782439199.1712662749,Y,N,17,20240411,Y,38.5,NaN,10,False
...,...,...,...,...,...,...,...,...,...,...
993,1854899250.1712454722,NaN,NaN,2205,20240407,N,99.6,out,2200,False
994,466793031.1712393578,NaN,NaN,2893,20240406,N,99.7,out,2890,False
995,2009790480.1712493801,NaN,NaN,3491,20240407,N,99.8,out,3490,False
996,218866954.1708578885,NaN,NaN,12349,20240410,N,99.9,out,12340,True


In [117]:
# Define the binary mapping with all expected values
binary_mapping = {'Y': 1, 'N': 0, 'in': 2, 'out': 3}

# Apply the mapping while filling NaN values
df['스크롤여부'] = df['스크롤여부'].map(binary_mapping).fillna(-1)
df['클릭여부'] = df['클릭여부'].map(binary_mapping).fillna(-1)
df['신청진입여부'] = df['신청진입여부'].map(binary_mapping).fillna(-1)
df['이탈유형'] = df['이탈유형'].map(binary_mapping).fillna(-1)


In [118]:
df

,아이디,스크롤여부,클릭여부,체류시간,date,신청진입여부,누적%,이탈유형,10초 구간,이상치
0,1042732193.1712370545,0.0,0.0,2,20240408,1,7.7,-1.0,0,False
1,1819785872.1712371735,0.0,0.0,10,20240406,1,15.4,-1.0,10,False
2,1653590084.1712352132,0.0,0.0,15,20240406,1,23.1,-1.0,10,False
3,2084490160.1712804830,1.0,0.0,17,20240411,1,30.8,-1.0,10,False
4,782439199.1712662749,1.0,0.0,17,20240411,1,38.5,-1.0,10,False
...,...,...,...,...,...,...,...,...,...,...
993,1854899250.1712454722,-1.0,-1.0,2205,20240407,0,99.6,3.0,2200,False
994,466793031.1712393578,-1.0,-1.0,2893,20240406,0,99.7,3.0,2890,False
995,2009790480.1712493801,-1.0,-1.0,3491,20240407,0,99.8,3.0,3490,False
996,218866954.1708578885,-1.0,-1.0,12349,20240410,0,99.9,3.0,12340,True


In [119]:
# Convert columns to appropriate data types
df['체류시간'] = pd.to_numeric(df['체류시간'], errors='coerce')
df['10초 구간'] = pd.to_numeric(df['10초 구간'], errors='coerce')

# Calculate the correlation matrix
correlation_matrix = df.corr()

print("""0.9 이상 또는 -0.9 이하: 매우 강한 상관관계
0.7 이상 또는 -0.7 이하: 강한 상관관계
0.5 이상 또는 -0.5 이하: 중간 정도의 상관관계
0.3 이상 또는 -0.3 이하: 약한 상관관계
0.1 이상 또는 -0.1 이하: 매우 약한 상관관계
0: 상관관계가 없음""")
      
correlation_matrix


0.9 이상 또는 -0.9 이하: 매우 강한 상관관계
0.7 이상 또는 -0.7 이하: 강한 상관관계
0.5 이상 또는 -0.5 이하: 중간 정도의 상관관계
0.3 이상 또는 -0.3 이하: 약한 상관관계
0.1 이상 또는 -0.1 이하: 매우 약한 상관관계
0: 상관관계가 없음


,아이디,스크롤여부,클릭여부,체류시간,date,신청진입여부,누적%,이탈유형,10초 구간,이상치
아이디,1.000000,0.029949,0.037757,0.017690,0.006304,0.030250,0.055684,-0.047026,0.017627,0.011348
스크롤여부,0.029949,1.000000,0.948050,0.355238,-0.023714,0.964323,0.035826,-0.848342,0.355169,0.420072
클릭여부,0.037757,0.948050,1.000000,0.307244,-0.023098,0.970356,0.022946,-0.853649,0.307204,0.443205
체류시간,0.017690,0.355238,0.307244,1.000000,-0.024314,0.310159,0.150065,-0.309167,0.999998,0.896333
date,0.006304,-0.023714,-0.023098,-0.024314,1.000000,-0.031798,-0.000830,0.028482,-0.024344,-0.002934
신청진입여부,0.030250,0.964323,0.970356,0.310159,-0.031798,1.000000,0.014906,-0.879727,0.310107,0.367341
누적%,0.055684,0.035826,0.022946,0.150065,-0.000830,0.014906,1.000000,-0.116410,0.148847,0.111243
이탈유형,-0.047026,-0.848342,-0.853649,-0.309167,0.028482,-0.879727,-0.116410,1.000000,-0.308959,-0.347607
10초 구간,0.017627,0.355169,0.307204,0.999998,-0.024344,0.310107,0.148847,-0.308959,1.000000,0.896350
이상치,0.011348,0.420072,0.443205,0.896333,-0.002934,0.367341,0.111243,-0.347607,0.896350,1.000000


In [120]:
# Define the threshold for strong correlation (absolute value of 0.7 or more)
threshold = 0.7

# Create a mask to identify correlations above the threshold
correlation_mask = correlation_matrix.abs() >= threshold

# Filter out correlations below the threshold
strong_correlations = correlation_matrix[correlation_mask].fillna('')

strong_correlations


,아이디,스크롤여부,클릭여부,체류시간,date,신청진입여부,누적%,이탈유형,10초 구간,이상치
아이디,1.0,,,,,,,,,
스크롤여부,,1.0,0.94805,,,0.964323,,-0.848342,,
클릭여부,,0.94805,1.0,,,0.970356,,-0.853649,,
체류시간,,,,1.0,,,,,0.999998,0.896333
date,,,,,1.0,,,,,
신청진입여부,,0.964323,0.970356,,,1.0,,-0.879727,,
누적%,,,,,,,1.0,,,
이탈유형,,-0.848342,-0.853649,,,-0.879727,,1.0,,
10초 구간,,,,0.999998,,,,,1.0,0.89635
이상치,,,,0.896333,,,,,0.89635,1.0


In [121]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

# Prepare the feature set (independent variables) and target variable (dependent variable)
# List of columns to retain as features
selected_features = ['스크롤여부', '체류시간', '누적%']
# Select only the specified columns as features
features = df[selected_features]
target = df['신청진입여부']

# Handle missing values
features_filled = features.fillna(-1)

# One-hot encoding for categorical features
features_encoded = pd.get_dummies(features_filled, drop_first=True)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_encoded, target, test_size=0.3, random_state=42)

# Standardize the feature data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize and train the logistic regression model
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train_scaled, y_train)

# Get the feature importance (coefficients) from the trained model
coefficients = model.coef_[0]

# Create a DataFrame to present the feature importance
feature_importance = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': coefficients
}).sort_values(by='Importance', ascending=False)

# Display the feature importance
print(feature_importance)


  Feature  Importance
0   스크롤여부    1.624961
1    체류시간   -0.001837
2     누적%   -0.288954


In [122]:
# Separate features based on the coefficient sign
positive_coefficients = feature_importance[feature_importance['Importance'] > 0]
negative_coefficients = feature_importance[feature_importance['Importance'] < 0]

# Display results
print("Features that increase the likelihood of '신청진입여부' being 1:")
print(positive_coefficients)

print("\nFeatures that increase the likelihood of '신청진입여부' being 0:")
print(negative_coefficients)

print("""Features that Increase the Likelihood of '신청진입여부' Being 1:
스크롤여부 (Importance = 1.586135)
스크롤여부 변수의 회귀 계수가 양수이므로, 스크롤을 사용한 방문자는 신청진입여부가 1이 될 가능성이 높습니다. 즉, 스크롤을 사용한 방문자는 특정 페이지에 진입할 가능성이 증가합니다.
Features that Increase the Likelihood of '신청진입여부' Being 0:
체류시간 (Importance = -0.010233)
체류시간 변수의 회귀 계수가 음수이므로, 체류 시간이 증가하면 신청진입여부가 0이 될 가능성이 높아집니다. 즉, 페이지에 오래 머물수록 신청 진입을 하지 않을 가능성이 높아집니다.
누적% (Importance = -0.130658)
누적% 변수의 회귀 계수도 음수이므로, 누적 백분율이 높아질수록 신청진입여부가 0이 될 가능성이 높습니다. 이는 페이지 방문을 거듭하며 누적된 방문 경험이 신청 진입을 하지 않을 가능성을 높인다는 것을 의미할 수 있습니다.""")

Features that increase the likelihood of '신청진입여부' being 1:
  Feature  Importance
0   스크롤여부    1.624961

Features that increase the likelihood of '신청진입여부' being 0:
  Feature  Importance
1    체류시간   -0.001837
2     누적%   -0.288954
Features that Increase the Likelihood of '신청진입여부' Being 1:
스크롤여부 (Importance = 1.586135)
스크롤여부 변수의 회귀 계수가 양수이므로, 스크롤을 사용한 방문자는 신청진입여부가 1이 될 가능성이 높습니다. 즉, 스크롤을 사용한 방문자는 특정 페이지에 진입할 가능성이 증가합니다.
Features that Increase the Likelihood of '신청진입여부' Being 0:
체류시간 (Importance = -0.010233)
체류시간 변수의 회귀 계수가 음수이므로, 체류 시간이 증가하면 신청진입여부가 0이 될 가능성이 높아집니다. 즉, 페이지에 오래 머물수록 신청 진입을 하지 않을 가능성이 높아집니다.
누적% (Importance = -0.130658)
누적% 변수의 회귀 계수도 음수이므로, 누적 백분율이 높아질수록 신청진입여부가 0이 될 가능성이 높습니다. 이는 페이지 방문을 거듭하며 누적된 방문 경험이 신청 진입을 하지 않을 가능성을 높인다는 것을 의미할 수 있습니다.


In [123]:
import pandas as pd

# Create a path or filename for the Excel file
excel_path = 'data_analysis_results_ql.xlsx'

# Use ExcelWriter to write each DataFrame into different sheets
with pd.ExcelWriter(excel_path) as writer:
    df.to_excel(writer, sheet_name='Original Data', index=False)  # Original data
    correlation_matrix.to_excel(writer, sheet_name='Correlation Matrix')  # Correlation matrix
    feature_importance.to_excel(writer, sheet_name='Feature Importance', index=False)  # Feature importance
